In [1]:
from GNEMS.GNEMS import GNEMS_segment
from PIL import Image
from PIL.PngImagePlugin import PngInfo
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
from skimage.color import label2rgb
from glob import glob
from tqdm import tqdm
import time
import treading

/Users/isaac/miniforge3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
method_name = f"GNEMS_alternate_{time.time()}"
!mkdir results/{method_name}
!mkdir results/{method_name}/noise
!mkdir results/{method_name}/noise/0.5
!mkdir results/{method_name}/noise/1.0
!mkdir results/{method_name}/noise/2.0
!mkdir results/{method_name}/noise/4.0
!mkdir results/{method_name}/noise/8.0
!mkdir results/{method_name}/clouds
!mkdir results/{method_name}/texture

def segment_and_save(path, bar=None):
    im = np.array(Image.open(path).convert("L"))
    d = 16
    seg = GNEMS_segment(im, d=d, iterations=50, prediction_stride=8, show_progress=False)
    category = path.split("/")[1]
    Image.fromarray(seg).convert("L").save(f"results/{method_name}/{category}/{id}.png")
    bar.update(1)

print("Segmenting clouds dataset...")
with open("datasets/clouds/test_ids.txt", "r") as f:
    test_ids = [line.strip() for line in f.readlines()]

threads = []
with tqdm(total = len(test_ids)) as pbar:
    for id in test_ids:
        path = f"datasets/clouds/images/{id}.png"
        t = threading.Thread(target=segment_and_save, args=(path,pbar))
        while len(threading.enumerate()) >= 256:
            time.sleep(1)
        t.start()
        threads.append(t)
    for t in threads:
        t.join()

with open("datasets/noise/test_ids.txt", "r") as f:
    test_ids = [line.strip() for line in f.readlines()]

print("Segmenting noise dataset...")
threads = []
with tqdm(total = len(test_ids)) as pbar:
    for id in test_ids:
        path = f"datasets/noise/{id}.png"
        t = threading.Thread(target=segment_and_save, args=(path,pbar))
        while len(threading.enumerate()) >= 256:
            time.sleep(1)
        t.start()
        threads.append(t)
    for t in threads:
        t.join()

print("Segmenting texture dataset...")
with open("datasets/texture/test_ids.txt", "r") as f:
    test_ids = [line.strip() for line in f.readlines()]

threads = []
with tqdm(total = len(test_ids)) as pbar:
    for id in test_ids:
        path = f"datasets/texture/images/{id}.png"
        t = threading.Thread(target=segment_and_save, args=(path,pbar))
        while len(threading.enumerate()) >= 256:
            time.sleep(1)
        t.start()
        threads.append(t)
    for t in threads:
        t.join()

  1%|          | 2/256 [00:07<15:04,  3.56s/it]


KeyboardInterrupt: 